In [1]:
import pickle
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
train_df = pd.read_csv("../../input/feedback-prize-english-language-learning/train.csv")
train_df.head(2)

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5


In [3]:
model_path_list = [
    #('../../14_Baseline4/exp/result/14_v1_01/oof_df.csv',1), # deberta-v3-base
    ('../../14_Baseline4/exp/result/14_v1_07/oof_df.csv',1), # deberta-v3-base, 10folds     
    #('../../14_Baseline4/exp/result/14_v1_08/oof_df.csv',1), # deberta-base
    #('../../14_Baseline4/exp/result/14_v1_09/oof_df.csv',1), # roberta-base
    ('../../14_Baseline4/exp/result/14_v1_10/oof_df.csv',1), # deberta-v3-large
    #('../../14_Baseline4/exp/result/14_v1_11/oof_df.csv',1), # xlm-roberta-base
    ('../../14_Baseline4/exp/result/14_v1_12/oof_df.csv',1), # deberta-large
    #('../../14_Baseline4/exp/result/14_v1_13/oof_df.csv',1), # roberta-large
    ('../../14_Baseline4/exp/result/14_v1_14/oof_df.csv',1), # xlm-roberta-large
#     ('../../14_Baseline4/exp/result/14_v1_15/oof_df.csv',1), # deberta-v2-xlarge
#     ('../../14_Baseline4/exp/result/14_v1_16/oof_df.csv',1), # deberta-xlarge
    
#     ('../../14_Baseline4/exp/result/14_v2_01/oof_df.csv',1), # deberta-v3-base, seed200
#     ('../../14_Baseline4/exp/result/14_v2_02/oof_df.csv',1), # deberta-v3-large, seed200
#     ('../../14_Baseline4/exp/result/14_v2_03/oof_df.csv',1), # deberta-base, seed200
    
    #('../../17_Pseudo3/exp/result/17_v1_01/oof_df.csv',1), # deberta-v3-base
    #('../../17_Pseudo3/exp/result/17_v1_02/oof_df.csv',1), # deberta-v3-base
    #('../../17_Pseudo3/exp/result/17_v1_03/oof_df.csv',1), # deberta-v3-large
    #('../../17_Pseudo3/exp/result/17_v1_04/oof_df.csv',1), # deberta-v3-base, seed200
    #('../../17_Pseudo3/exp/result/17_v1_05/oof_df.csv',1), # deberta-base
    #('../../17_Pseudo3/exp/result/17_v1_06/oof_df.csv',1), # deberta-large
    #('../../17_Pseudo3/exp/result/17_v1_07/oof_df.csv',1), # deberta-v3-small
    
    #('../../18_Pseudo4/exp/result/18_v1_01/oof_df.csv',1), # deberta-v3-base
    #('../../18_Pseudo4/exp/result/18_v1_02/oof_df.csv',1), # deberta-v3-large
    #('../../18_Pseudo4/exp/result/18_v1_03/oof_df.csv',1), # deberta-v3-small
    
    #('../../19_Distillation/exp/result/19_v1_01/oof_df.csv',1), # deberta-v3-base <- dbv3-l
    ('../../19_Distillation/exp/result/19_v1_02/oof_df.csv',1), # deberta-v3-base <- dbv3-xl
    ('../../19_Distillation/exp/result/19_v1_03/oof_df.csv',1), # deberta-v3-base <- db-xl
    #('../../19_Distillation/exp/result/19_v1_04/oof_df.csv',1), # deberta-v3-small <- dbv3-l
    #('../../19_Distillation/exp/result/19_v1_05/oof_df.csv',1), # deberta-v3-base <- db-l
    
    # single models
    ('../../20_Distillation2/exp/result/20_v2_01/oof_df.csv',1), # deberta-v3-base <- dbv3-b
    #('../../20_Distillation2/exp/result/20_v2_02/oof_df.csv',1), # deberta-v3-base <- dbv3-l
    #('../../20_Distillation2/exp/result/20_v2_03/oof_df.csv',1), # deberta-v3-large <- dbv3-l
    #('../../20_Distillation2/exp/result/20_v2_04/oof_df.csv',1), # deberta-base <- db-b
    #('../../20_Distillation2/exp/result/20_v2_05/oof_df.csv',1), # deberta-v3-base <- db-b
    #('../../20_Distillation2/exp/result/20_v2_06/oof_df.csv',1), # deberta-v3-base <- db-l
]

oof_df_list = [
    pd.read_csv(model_path) for model_path,_ in model_path_list
]
weights_list = [
    w for _,w in model_path_list
]
weights_list = [w/sum(weights_list) for w in weights_list]

for oof_df in oof_df_list:
    oof_df = train_df[['text_id']].merge(oof_df, how='left', on='text_id')
    
num_models = len(model_path_list)

TARGET_COLS = ['cohesion','syntax','vocabulary','phraseology','grammar','conventions']

preds = 0
for oof_df, w in zip(oof_df_list, weights_list):
    preds += oof_df[TARGET_COLS].values * w

oofs = []
for oof_df in oof_df_list:
    oofs.append(oof_df[TARGET_COLS].values)

In [4]:
oof_df = oof_df_list[0].copy()
for i,col in enumerate(TARGET_COLS):
    oof_df[col] = preds[:,i]

In [5]:
import numpy as np

def calc_metric(pred, gt):
    '''
    pred : (num_data, num_labels)
    gt : (num_data, num_labels)
    '''
    score = np.sqrt(np.mean((pred - gt)**2, axis=0))
    score = score.mean()
    return score

In [6]:
score = calc_metric(pred=oof_df[TARGET_COLS].values, gt=train_df[TARGET_COLS].values)
print('CV={:.4f}'.format(score))

CV=0.4434


# Optimize class-wise

In [7]:
ensemble_predictions=np.stack(oofs)
ensemble_predictions.shape

(7, 3911, 6)

In [8]:
from skopt import gp_minimize

def run_optimize(i):
    target_col = TARGET_COLS[i]
    print("*"*50)
    print(target_col)
    print("*"*50)
    def ensemble_score(weights,return_pred=False):
        weights=np.array(weights)
        weights=weights.reshape(-1,1,1)/weights.sum()
        p=weights.reshape(-1,1,1)*ensemble_predictions[:,:,i,None]
        p=p.sum(0)
        score=calc_metric(p, train_df[[target_col]])
        if return_pred:
            return score,p
        else:
            return score   
    results = gp_minimize(ensemble_score, np.array([[0.1,1] for i in range(len(ensemble_predictions))]),
                          verbose=True,random_state=2022)
    best_weights=np.array(results['x'])/sum(results['x'])
    print(best_weights)
    # smaller is better for this metric
    score, ensemble_pred=ensemble_score(best_weights,True)
    print("score={:.4f}", score)
    return score, best_weights

In [9]:
scores = {}
best_weights = {}
for i,target_col in enumerate(TARGET_COLS):
    score, weights = run_optimize(i)
    scores[target_col] = score
    best_weights[target_col] = weights

**************************************************
cohesion
**************************************************
Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.0046
Function value obtained: 0.4717
Current minimum: 0.4717
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.0021
Function value obtained: 0.4714
Current minimum: 0.4714
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 0.0018
Function value obtained: 0.4732
Current minimum: 0.4714
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 0.0017
Function value obtained: 0.4715
Current minimum: 0.4714
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.

Iteration No: 41 ended. Search finished for the next optimal point.
Time taken: 0.3987
Function value obtained: 0.4711
Current minimum: 0.4710
Iteration No: 42 started. Searching for the next optimal point.
Iteration No: 42 ended. Search finished for the next optimal point.
Time taken: 0.4440
Function value obtained: 0.4724
Current minimum: 0.4710
Iteration No: 43 started. Searching for the next optimal point.
Iteration No: 43 ended. Search finished for the next optimal point.
Time taken: 0.4266
Function value obtained: 0.4724
Current minimum: 0.4710
Iteration No: 44 started. Searching for the next optimal point.
Iteration No: 44 ended. Search finished for the next optimal point.
Time taken: 0.4479
Function value obtained: 0.4727
Current minimum: 0.4710
Iteration No: 45 started. Searching for the next optimal point.
Iteration No: 45 ended. Search finished for the next optimal point.
Time taken: 0.4409
Function value obtained: 0.4722
Current minimum: 0.4710
Iteration No: 46 started. Sea

Iteration No: 81 ended. Search finished for the next optimal point.
Time taken: 0.5809
Function value obtained: 0.4718
Current minimum: 0.4709
Iteration No: 82 started. Searching for the next optimal point.
Iteration No: 82 ended. Search finished for the next optimal point.
Time taken: 0.6102
Function value obtained: 0.4709
Current minimum: 0.4709
Iteration No: 83 started. Searching for the next optimal point.
Iteration No: 83 ended. Search finished for the next optimal point.
Time taken: 0.6082
Function value obtained: 0.4717
Current minimum: 0.4709
Iteration No: 84 started. Searching for the next optimal point.
Iteration No: 84 ended. Search finished for the next optimal point.
Time taken: 0.6270
Function value obtained: 0.4717
Current minimum: 0.4709
Iteration No: 85 started. Searching for the next optimal point.
Iteration No: 85 ended. Search finished for the next optimal point.
Time taken: 0.6294
Function value obtained: 0.4709
Current minimum: 0.4709
Iteration No: 86 started. Sea

Iteration No: 21 ended. Search finished for the next optimal point.
Time taken: 0.3048
Function value obtained: 0.4379
Current minimum: 0.4364
Iteration No: 22 started. Searching for the next optimal point.
Iteration No: 22 ended. Search finished for the next optimal point.
Time taken: 0.2670
Function value obtained: 0.4368
Current minimum: 0.4364
Iteration No: 23 started. Searching for the next optimal point.
Iteration No: 23 ended. Search finished for the next optimal point.
Time taken: 0.2600
Function value obtained: 0.4378
Current minimum: 0.4364
Iteration No: 24 started. Searching for the next optimal point.
Iteration No: 24 ended. Search finished for the next optimal point.
Time taken: 0.2703
Function value obtained: 0.4377
Current minimum: 0.4364
Iteration No: 25 started. Searching for the next optimal point.
Iteration No: 25 ended. Search finished for the next optimal point.
Time taken: 0.2932
Function value obtained: 0.4369
Current minimum: 0.4364
Iteration No: 26 started. Sea

Iteration No: 61 ended. Search finished for the next optimal point.
Time taken: 0.4582
Function value obtained: 0.4371
Current minimum: 0.4363
Iteration No: 62 started. Searching for the next optimal point.
Iteration No: 62 ended. Search finished for the next optimal point.
Time taken: 0.4466
Function value obtained: 0.4375
Current minimum: 0.4363
Iteration No: 63 started. Searching for the next optimal point.
Iteration No: 63 ended. Search finished for the next optimal point.
Time taken: 0.5291
Function value obtained: 0.4371
Current minimum: 0.4363
Iteration No: 64 started. Searching for the next optimal point.
Iteration No: 64 ended. Search finished for the next optimal point.
Time taken: 0.5114
Function value obtained: 0.4378
Current minimum: 0.4363
Iteration No: 65 started. Searching for the next optimal point.
Iteration No: 65 ended. Search finished for the next optimal point.
Time taken: 0.5168
Function value obtained: 0.4376
Current minimum: 0.4363
Iteration No: 66 started. Sea

...

Iteration No: 11 ended. Search finished for the next optimal point.
Time taken: 0.2017
Function value obtained: 0.4647
Current minimum: 0.4630
Iteration No: 12 started. Searching for the next optimal point.
Iteration No: 12 ended. Search finished for the next optimal point.
Time taken: 0.2452
Function value obtained: 0.4631
Current minimum: 0.4630
Iteration No: 13 started. Searching for the next optimal point.
Iteration No: 13 ended. Search finished for the next optimal point.
Time taken: 0.3069
Function value obtained: 0.4628
Current minimum: 0.4628
Iteration No: 14 started. Searching for the next optimal point.
Iteration No: 14 ended. Search finished for the next optimal point.
Time taken: 0.2201
Function value obtained: 0.4630
Current minimum: 0.4628
Iteration No: 15 started. Searching for the next optimal point.
Iteration No: 15 ended. Search finished for the next optimal point.
Time taken: 0.3158
Function value obtained: 0.4627
Current minimum: 0.4627
Iteration No: 16 started. Sea

Iteration No: 51 ended. Search finished for the next optimal point.
Time taken: 0.4265
Function value obtained: 0.4645
Current minimum: 0.4622
Iteration No: 52 started. Searching for the next optimal point.
Iteration No: 52 ended. Search finished for the next optimal point.
Time taken: 0.4023
Function value obtained: 0.4644
Current minimum: 0.4622
Iteration No: 53 started. Searching for the next optimal point.
Iteration No: 53 ended. Search finished for the next optimal point.
Time taken: 0.3895
Function value obtained: 0.4628
Current minimum: 0.4622
Iteration No: 54 started. Searching for the next optimal point.
Iteration No: 54 ended. Search finished for the next optimal point.
Time taken: 0.4305
Function value obtained: 0.4622
Current minimum: 0.4622
Iteration No: 55 started. Searching for the next optimal point.
Iteration No: 55 ended. Search finished for the next optimal point.
Time taken: 0.4350
Function value obtained: 0.4623
Current minimum: 0.4622
Iteration No: 56 started. Sea

Iteration No: 91 ended. Search finished for the next optimal point.
Time taken: 0.7178
Function value obtained: 0.4630
Current minimum: 0.4622
Iteration No: 92 started. Searching for the next optimal point.
Iteration No: 92 ended. Search finished for the next optimal point.
Time taken: 0.7130
Function value obtained: 0.4622
Current minimum: 0.4622
Iteration No: 93 started. Searching for the next optimal point.
Iteration No: 93 ended. Search finished for the next optimal point.
Time taken: 0.7637
Function value obtained: 0.4635
Current minimum: 0.4622
Iteration No: 94 started. Searching for the next optimal point.
Iteration No: 94 ended. Search finished for the next optimal point.
Time taken: 0.7226
Function value obtained: 0.4622
Current minimum: 0.4622
Iteration No: 95 started. Searching for the next optimal point.
Iteration No: 95 ended. Search finished for the next optimal point.
Time taken: 0.7686
Function value obtained: 0.4622
Current minimum: 0.4622
Iteration No: 96 started. Sea

Iteration No: 31 ended. Search finished for the next optimal point.
Time taken: 0.2677
Function value obtained: 0.4369
Current minimum: 0.4368
Iteration No: 32 started. Searching for the next optimal point.
Iteration No: 32 ended. Search finished for the next optimal point.
Time taken: 0.2872
Function value obtained: 0.4370
Current minimum: 0.4368
Iteration No: 33 started. Searching for the next optimal point.
Iteration No: 33 ended. Search finished for the next optimal point.
Time taken: 0.3096
Function value obtained: 0.4370
Current minimum: 0.4368
Iteration No: 34 started. Searching for the next optimal point.
Iteration No: 34 ended. Search finished for the next optimal point.
Time taken: 0.3361
Function value obtained: 0.4368
Current minimum: 0.4368
Iteration No: 35 started. Searching for the next optimal point.
Iteration No: 35 ended. Search finished for the next optimal point.
Time taken: 0.3787
Function value obtained: 0.4369
Current minimum: 0.4368
Iteration No: 36 started. Sea

Iteration No: 71 ended. Search finished for the next optimal point.
Time taken: 0.5948
Function value obtained: 0.4374
Current minimum: 0.4367
Iteration No: 72 started. Searching for the next optimal point.
Iteration No: 72 ended. Search finished for the next optimal point.
Time taken: 0.6277
Function value obtained: 0.4367
Current minimum: 0.4367
Iteration No: 73 started. Searching for the next optimal point.
Iteration No: 73 ended. Search finished for the next optimal point.
Time taken: 0.5316
Function value obtained: 0.4369
Current minimum: 0.4367
Iteration No: 74 started. Searching for the next optimal point.
Iteration No: 74 ended. Search finished for the next optimal point.
Time taken: 0.5460
Function value obtained: 0.4371
Current minimum: 0.4367
Iteration No: 75 started. Searching for the next optimal point.
Iteration No: 75 ended. Search finished for the next optimal point.
Time taken: 0.5360
Function value obtained: 0.4370
Current minimum: 0.4367
Iteration No: 76 started. Sea

In [10]:
best_weights

{'cohesion': array([0.28681732, 0.13608497, 0.26068645, 0.08900075, 0.12645889,
        0.07226989, 0.02868173]),
 'syntax': array([0.23235636, 0.1724995 , 0.23235636, 0.15282973, 0.16348677,
        0.02323564, 0.02323564]),
 'vocabulary': array([0.23152612, 0.28505892, 0.14640456, 0.11301948, 0.05506889,
        0.02850589, 0.14041614]),
 'phraseology': array([0.21261176, 0.19602073, 0.16757288, 0.11208751, 0.12983941,
        0.04652298, 0.13534472]),
 'grammar': array([0.23258228, 0.37193479, 0.03719348, 0.03719348, 0.10380119,
        0.11520607, 0.10208871]),
 'conventions': array([0.23628938, 0.12993775, 0.16048939, 0.12349143, 0.15886405,
        0.16729907, 0.02362894])}

In [11]:
scores

{'cohesion': 0.4708578310209242,
 'syntax': 0.43626479151323333,
 'vocabulary': 0.40494345148818756,
 'phraseology': 0.44575307875424736,
 'grammar': 0.46215371865139815,
 'conventions': 0.43669756134814464}

In [12]:
CV = sum(scores.values()) / len(scores)
print("CV={:.4f}".format(CV))

CV=0.4428


# Error Analysis - Check Corr

In [13]:
cols = ['cohesion','syntax','vocabulary','phraseology','grammar','conventions']
train_df[cols].corr()

,cohesion,syntax,vocabulary,phraseology,grammar,conventions
cohesion,1.000000,0.695459,0.666151,0.690058,0.638689,0.666151
syntax,0.695459,1.000000,0.680562,0.725467,0.709525,0.700025
vocabulary,0.666151,0.680562,1.000000,0.735261,0.654852,0.664292
phraseology,0.690058,0.725467,0.735261,1.000000,0.719746,0.666842
grammar,0.638689,0.709525,0.654852,0.719746,1.000000,0.673301
conventions,0.666151,0.700025,0.664292,0.666842,0.673301,1.000000


In [14]:
cols = ['cohesion','syntax','vocabulary','phraseology','grammar','conventions']
oof_df[cols].corr()

,cohesion,syntax,vocabulary,phraseology,grammar,conventions
cohesion,1.000000,0.950083,0.920389,0.916340,0.830124,0.891026
syntax,0.950083,1.000000,0.944059,0.965599,0.924438,0.931105
vocabulary,0.920389,0.944059,1.000000,0.963558,0.882401,0.898596
phraseology,0.916340,0.965599,0.963558,1.000000,0.947073,0.889407
grammar,0.830124,0.924438,0.882401,0.947073,1.000000,0.861789
conventions,0.891026,0.931105,0.898596,0.889407,0.861789,1.000000
